# Imports

In [ ]:
import wandb
from llm_non_identifiability.runner import LightningGrammarModule
from llm_non_identifiability.datamodule import GrammarDataModule

from pytorch_lightning import Trainer
from os.path import abspath, dirname, join
import torch

# Load model

## Download artifact from wandb

In [ ]:

run = wandb.init()
model_name = 'model-nifu9qxw:v14'
artifact = run.use_artifact(f'causal-representation-learning/llm-non-identifiability/{model_name}', type='model')
artifact_dir = artifact.download()

## Load model from checkpoint

In [ ]:
# source: https://pytorch-lightning.readthedocs.io/en/stable/common/checkpointing.html#checkpoint-loading
checkpoint = "model.ckpt"
PATH = join(artifact_dir, checkpoint)

# To load a model along with its weights and hyperparameters use the following method
model: LightningGrammarModule = LightningGrammarModule.load_from_checkpoint(
    PATH, device="cuda" if torch.cuda.is_available() else "cpu"
)